# Project

## Packages

In [1]:
from bs4 import BeautifulSoup # to parse external data
import yfinance as yf
import pandas as pd # to read CSV files
import requests # to get data
import spacy #to extraxt entities

## RSS feeds

In [2]:
# Check if data from the page are downloadable
yahoo = requests.get("https://finance.yahoo.com/news/rssindex")
yahoo

<Response [200]>

In [3]:
# Get headlines
Tyahoo = BeautifulSoup(yahoo.content, features='xml')
TY = Tyahoo.findAll('title')
TY # list of titles

[<title>Yahoo Finance</title>,
 <title>Yahoo Finance</title>,
 <title>Eight High Dividend Stocks You Can Count On</title>,
 <title>Jim Cramer literally drowns Cathie Wood’s ARK as flagship fund falters</title>,
 <title>RMDs May Soon Start Even Later for Retirement Plan Savers</title>,
 <title>Tech Is Still Getting Crushed. Here Are 15 Stocks to Buy in a Changed World.</title>,
 <title>‘Easter Bunny cartoon cash’ — how Bill Maher called the Bitcoin crash</title>,
 <title>Intel stock hits lowest price in more than a year, and there is another shoe to drop</title>,
 <title>Tesla rival Rivian's stock could skyrocket at least 160%, says top analyst</title>,
 <title>Apple could have another blowout year amid ‘scary’ demand for its products</title>,
 <title>Top Dividend Stocks for February 2022</title>,
 <title>Fidelity Joins Invesco and Schwab in Adding China Warnings to Funds</title>,
 <title>Is Disney Stock A Buy Right Now, As It Keeps Sinking?</title>,
 <title>Top Growth Stocks for Februa

In [4]:
wsj = requests.get("https://feeds.a.dj.com/rss/RSSMarketsMain.xml?fbclid=IwAR17gY8vV2SdoTLP_35v7zGYmPireg5xIX_y1VEgPYRoXVd5jVouoKRlXAc")
wsj

<Response [200]>

In [5]:
Twsj = BeautifulSoup(wsj.content, features='xml')
TW = Twsj.findAll('title')
TW

[<title>WSJ.com: Markets</title>,
 <title>WSJ.com: Markets</title>,
 <title>Who Really Got Rich From the GameStop Revolution?</title>,
 <title>Whirlpool, Kroger, Apple: Stocks That Defined the Week</title>,
 <title>S&amp;P 500 Rises, Snaps Three-Week Losing Streak</title>,
 <title>Goldman Doubles David Solomon's Pay to $35 Million</title>,
 <title>Robinhood Shares Swing in Volatile Session After Earnings Miss</title>,
 <title>Chevron Still Has Gas in Its Tank</title>,
 <title>SoftBank Operating Chief Marcelo Claure to Leave After Pay Dispute</title>,
 <title>The Two Things to Do When the Stock Market Gets Crazy</title>,
 <title>Elliott Management to Sell Stake in Hong Kong Bank, Ending Activist Campaign</title>,
 <title>Steel Market Cools as Supplies Expand</title>,
 <title>Hedge Fund Melvin Lost $6.8 Billion in a Month. Winning It Back Is Taking a Lot Longer.</title>,
 <title>Are Foreign Investors in China Second Class Citizens Now?</title>,
 <title>Apple, Robinhood, Visa, Western Dig

In [6]:
cnbc = requests.get("https://www.cnbc.com/id/15839135/device/rss/rss.html?fbclid=IwAR2o0zeWtmgEwZob45_F6e02pkTVo9uBGL0VI1GQv8mPyScEFY-hn9t089Y")
cnbc

<Response [200]>

In [7]:
Tcnbc = BeautifulSoup(cnbc.content, features='xml')
TC = Tcnbc.findAll('title')
TC

[<title>Earnings</title>,
 <title>Chevron shares retreat from a record after fourth-quarter profit falls short of expectations</title>,
 <title>Atlassian jumps as quarterly results top estimates and company raises guidance for 2022 subscriptions</title>,
 <title>Apple revenue pops 11% to $123.9 billion, Cook says supply chain improving</title>,
 <title>Robinhood loses active users in the fourth quarter, forecasts weak revenue</title>,
 <title>Southwest Airlines says omicron will drive a first-quarter loss but expects 2022 profits</title>,
 <title>Comcast beats earnings expectations, but falls short on new internet customers</title>,
 <title>McDonald's earnings miss estimates as rising costs weigh on profits</title>,
 <title>Deutsche Bank posts a surprise profit on strong investment bank performance</title>,
 <title>Beaten-down cloud software stocks get boost as ServiceNow and Qualtrics top estimates</title>,
 <title>Tesla beats on earnings and revenue, says supply chain issues were 'ma

## Extract entities

In [8]:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm") # en_core_web_sm - basic NLP task (to process extracted text data)

In [9]:
processed_hline = nlp(TC[1].text) #Get first headline
print(TC[1])
for token in processed_hline:
    print(token.text, "-----", spacy.explain(token.pos_),'-----', spacy.explain(token.dep_)) # tokens + tags + dependencies

<title>Chevron shares retreat from a record after fourth-quarter profit falls short of expectations</title>
Chevron ----- proper noun ----- compound
shares ----- noun ----- nominal subject
retreat ----- verb ----- None
from ----- adposition ----- prepositional modifier
a ----- determiner ----- determiner
record ----- noun ----- object of preposition
after ----- subordinating conjunction ----- marker
fourth ----- adjective ----- adjectival modifier
- ----- punctuation ----- punctuation
quarter ----- noun ----- compound
profit ----- noun ----- nominal subject
falls ----- verb ----- adverbial clause modifier
short ----- adjective ----- adverbial modifier
of ----- adposition ----- prepositional modifier
expectations ----- noun ----- object of preposition


In [10]:
# See the dependencies
spacy.displacy.render(processed_hline, style='dep', jupyter=True, options={'distance': 110}) 

In [11]:
# Show important entities; tag ORG = Companies
spacy.displacy.render(processed_hline, style='ent', jupyter=True, options={'distance': 120}) 

In [12]:
# Extract ORG from headlines
companies = []
for title in TC:
    doc = nlp(title.text)
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else: 
            pass
        
companies

['Chevron',
 'Apple',
 'Southwest Airlines',
 "McDonald's",
 'Deutsche Bank',
 'ServiceNow',
 'Qualtrics',
 'Intel',
 'Boeing',
 'Microsoft',
 'J&J',
 'IBM',
 'American Airlines',
 'United',
 'P&G',
 'Morgan Stanley',
 'Bank of America',
 'Goldman',
 "Wells Fargo's",
 'Citigroup',
 'JPMorgan',
 'Apple',
 'TSMC',
 "Bed Bath & Beyond's",
 'Nike']

In [13]:
stocks_df = pd.read_csv('./SP500.csv') ## https://github.com/datasets/s-and-p-500-companies
stocks_df.head() 

,Symbol,Sector,Name
0,MMM,Industrials,3M
1,AOS,Industrials,A. O. Smith
2,ABT,Health Care,Abbott
3,ABBV,Health Care,AbbVie
4,ABMD,Health Care,Abiomed


In [14]:
# attributes of stocks
stock_dict = {
    'Org': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    'forwardPE': [],
    'dividendYield': []
}

# collecting information
for company in companies:
    try:
        if stocks_df['Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Name'].\
                                str.contains(company)]['Name'].values[0]
            stock_dict['Org'].append(org_name)
            stock_dict['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_dict['currentPrice'].append(stock_info['currentPrice'])
            stock_dict['dayHigh'].append(stock_info['dayHigh'])
            stock_dict['dayLow'].append(stock_info['dayLow'])
            stock_dict['forwardPE'].append(stock_info['forwardPE'])
            stock_dict['dividendYield'].append(stock_info['dividendYield'])
        else:
            pass
    except:
        pass

# dataframe
pd.DataFrame(stock_dict)

,Org,Symbol,currentPrice,dayHigh,dayLow,forwardPE,dividendYield
0,Chevron,CVX,130.61,132.0600,128.070,12.996019,0.0435
1,Apple,AAPL,170.33,170.3500,162.800,27.472582,0.0052
2,Southwest Airlines,LUV,42.93,42.9600,40.630,11.991620,NaN
3,McDonald's,MCD,256.09,256.3453,247.160,23.071170,0.0222
4,ServiceNow,NOW,561.08,563.4979,507.800,60.201720,NaN
5,Intel,INTC,47.73,48.2000,46.300,12.830645,0.0291
6,Boeing,BA,190.57,190.7200,183.770,25.308102,NaN
7,Microsoft,MSFT,308.26,308.4800,294.450,28.809347,0.0080
8,IBM,IBM,134.50,134.5300,131.790,12.858508,0.0495
9,American Airlines Group,AAL,15.64,15.7750,14.905,7.629269,NaN
